In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
import regex as re
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import distance
from fuzzywuzzy import fuzz

In [44]:
df=pd.read_csv(r"C:\Users\Johaa\Desktop\ML PROJECTS\DUPLICATE QUESTION PAIRS\train.csv")
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [45]:
df.dropna(inplace=True)

In [46]:
df=df.sample(30000)

In [47]:
df

,id,qid1,qid2,question1,question2,is_duplicate
235234,235234,345908,345909,I read that divorce rates in Iran are increasi...,Are there any significant differences in the d...,0
322679,322679,266746,448533,How do I focus?,What should I do to focus on my education?,0
355018,355018,41705,205197,Why did RBI issue new 2000 rupees note?,Why is the RBI issuing 2000 rupee notes?,1
44788,44788,80345,80346,How can the density of a sphere formula be cal...,What is the formula for the density of a sphere?,0
312226,312226,73626,294927,Why do stars twinkle?,Why we can feel stars twinkling?,1
...,...,...,...,...,...,...
30494,30494,56310,56311,Is there any truth to the Rothschild MH370 con...,"Are there any truth to these so called ""conspi...",0
263208,263208,12591,28646,What's the sole purpose of life?,What do you feel is your purpose in life?,1
283052,283052,86534,403086,What has been the single most important life d...,What is the biggest/most important decision yo...,1
176945,176945,272126,272127,Will China replace the United States as the wo...,Will the United States of America surrive the ...,0


## MUCH BETTER TEXT PREPROCESSING

In [48]:
def preprocessing(text):
    text=text.lower().strip()#lowercasing our text and removing any leading and trailing white spaces
    
    #replacing some symbols with their textual equivalent
    text=text.replace("%","percent")
    text=text.replace("$","dollar")
    text=text.replace("@","at")
    text=text.replace("€","euro")
    text=text.replace("₹","rupee")
    
    #replacing [math] with "". it doesn't have any meaning
    text=text.replace("[math]","")
    
    #replacing numbers with their string equivalent
    text=text.replace(",000,000,000 ","b")
    text=text.replace(",000,000 ","m")
    text=text.replace(",000 ","k")
    
    text=re.sub(r"([0-9]+)000000000",r"\1b",text)
    text=re.sub(r"([0-9]+)000000",r"\1m",text)
    text=re.sub(r"([0-9]+)000",r"\1k",text)
    
    #Decontracting words
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }
    
    text_decontracted=[]
    
    for word in text.split():
        if word in contractions:
            word=contractions[word]
        
        text_decontracted.append(word)
    
    text=" ".join(text_decontracted)
    
    text=text.replace("'ve"," have")
    text=text.replace("'re"," are")
    text=text.replace("'ll"," will")
    text=text.replace("n't"," not")
    
    #removing non-word characters(puctuation marks)
    pattern=re.compile("\W")
    text=re.sub(pattern," ",text)
    
    #removing URLS
    pattern=r"https?://\S"
    text=re.sub(pattern,"",text)
    
    #removing HTML tags
    bo=BeautifulSoup(text)
    text=bo.get_text()
    
    lemmatizer=WordNetLemmatizer()
    temp_text=[]
    for word in text.split():
        temp_text.append(lemmatizer.lemmatize(word,pos="v"))
    
    text=" ".join(temp_text)
    
    
    return text
    

In [49]:
df["question1"]=df["question1"].apply(preprocessing)
df["question2"]=df["question2"].apply(preprocessing)

In [50]:
df

,id,qid1,qid2,question1,question2,is_duplicate
235234,235234,345908,345909,i read that divorce rat in iran be increase do...,be there any significant differences in the di...,0
322679,322679,266746,448533,how do i focus,what should i do to focus on my education,0
355018,355018,41705,205197,why do rbi issue new 2k rupees note,why be the rbi issue 2k rupee note,1
44788,44788,80345,80346,how can the density of a sphere formula be cal...,what be the formula for the density of a sphere,0
312226,312226,73626,294927,why do star twinkle,why we can feel star twinkle,1
...,...,...,...,...,...,...
30494,30494,56310,56311,be there any truth to the rothschild mh370 con...,be there any truth to these so call conspiracy...,0
263208,263208,12591,28646,what be the sole purpose of life,what do you feel be your purpose in life,1
283052,283052,86534,403086,what have be the single most important life de...,what be the biggest most important decision yo...,1
176945,176945,272126,272127,will china replace the unite state as the worl...,will the unite state of america surrive the lo...,0


In [51]:
def common_words(row):
    w1=set(map(lambda word: word.lower().strip(),row["question1"].split()))
    w2=set(map(lambda word: word.lower().strip(),row["question2"].split()))
    
    return len(w1&w2)
def total_words(row):
    w1=set(map(lambda word: word.lower().strip(),row["question1"].split()))
    w2=set(map(lambda word: word.lower().strip(),row["question2"].split()))
    
    return len(w1) +len(w2)
df["q1_len"]=df["question1"].str.len()
df["q2_len"]=df["question2"].str.len()
df["q2_words"]=df["question2"].apply(lambda x: len(x.split()))
df["q1_words"]=df["question1"].apply(lambda x: len (x.split()))
df["word_common"]=df.apply(common_words,axis=1)
df["total_words"]=df.apply(total_words,axis=1)
df["word_share"]=round(df["word_common"]/df["total_words"],2)

## CREATING MUCH ADVANCED FEATURES

### TOKEN FEATURES

In [52]:
def fetch_token_features(row):
    
    q1=row["question1"]
    q2=row["question2"]
    
    STOPWORDS=stopwords.words("English")
    safe_div=0.00001
    
    #tokenizing both q1 and q2
    q1_tokens=q1.split()
    q2_tokens=q2.split()
    
    token_features=[0.0]*8
    
    if len(q1_tokens)==0 or len(q2_tokens)==0:
        return token_features
    
    #storing stop words of both q1 and q2
    q1_stopwords=set([word for word in q1_tokens if word in STOPWORDS])
    q2_stopwords=set([word for word in q2_tokens if word in STOPWORDS])
    
    #storing non-stopwords of both q1 and q2
    q1_words=set([word for word in q1_tokens if word not in STOPWORDS])
    q2_words=set([word for word in q2_tokens if word not in STOPWORDS])
    
    #common stop words
    common_stopwords=len(q1_stopwords&q2_stopwords)
    
    #common tokens
    common_tokens=len(set(q1_tokens)&set(q2_tokens))
    
    #common non stop words
    common_words=len(q1_words&q2_words)
    
    token_features[0]=common_words/(min(len(q1_words),len(q2_words))+safe_div)
    token_features[1]=common_words/(max(len(q1_words),len(q2_words))+safe_div)
    token_features[2]=common_stopwords/(min(len(q1_stopwords),len(q2_stopwords))+safe_div)
    token_features[3]=common_stopwords/(max(len(q1_stopwords),len(q2_stopwords))+safe_div)
    token_features[4]=common_tokens/(min(len(q1_tokens),len(q2_tokens))+safe_div)
    token_features[5]=common_tokens/(max(len(q1_tokens),len(q2_tokens))+safe_div)
    
    #checking if the last and first word are the same
    token_features[6]=int(q1_tokens[-1]==q2_tokens[-1])
    token_features[7]=int(q1_tokens[0]==q2_tokens[0])
    
    return token_features
    

In [53]:
token_features=df.apply(fetch_token_features,axis=1)

In [54]:
new_df=pd.DataFrame(token_features.to_list(),columns=["cwc_min","cwc_max","csw_min","csw_max","cmt_min","cmt_max","last_word_equal","first_word_equal"],index=df.index)

In [55]:
new_df

,cwc_min,cwc_max,csw_min,csw_max,cmt_min,cmt_max,last_word_equal,first_word_equal
235234,0.250000,0.250000,0.571428,0.499999,0.375000,0.315789,0.0,0.0
322679,0.999990,0.499998,0.666664,0.285714,0.749998,0.333333,0.0,0.0
355018,0.799998,0.666666,0.499998,0.333332,0.624999,0.624999,1.0,1.0
44788,0.999997,0.749998,0.666666,0.666666,0.699999,0.699999,0.0,0.0
312226,0.999995,0.666664,0.499998,0.333332,0.749998,0.499999,1.0,1.0
...,...,...,...,...,...,...,...,...
30494,0.399999,0.399999,0.799998,0.363636,0.599999,0.352941,0.0,1.0
263208,0.666664,0.666664,0.499999,0.333333,0.571428,0.444444,1.0,1.0
283052,0.499999,0.375000,0.714285,0.624999,0.533333,0.500000,0.0,1.0
176945,0.444444,0.444444,0.499999,0.399999,0.400000,0.375000,0.0,1.0


### LENGTH BASED FEATURES

In [56]:
list(distance.lcsubstrings("be hello most users oh of quora indians justindian","how many quora hello users be indians oh justindian"))[0]

' justindian'

In [57]:
def length_based_features(row):
    q1=row["question1"]
    q2=row["question2"]
    
    length_features=[0.0]*3
    
    q1_tokens=q1.split()
    q2_tokens=q2.split()
    
    if len(q1_tokens)==0 or len(q2_tokens)==0:
        return length_features
    length_features[0]=(len(q1_tokens)+len(q2_tokens))/2
    length_features[1]=abs(len(q1_tokens)-len(q2_tokens))
    
    substring=list(distance.lcsubstrings(q1,q2))
    if len(substring)==0:
        length_features[2]==0
    else:
        length_features[2]=len(substring[0])/(min(len(q1),len(q2))+1)
    
    return length_features
    

In [58]:
length_features=df.apply(length_based_features,axis=1)

In [59]:
length_features_df=pd.DataFrame(length_features.to_list(),columns=["mean_length","abs_length","longest_substr_ratio"],index=df.index)

In [60]:
length_features_df

,mean_length,abs_length,longest_substr_ratio
235234,17.5,3.0,0.160494
322679,6.5,5.0,0.400000
355018,8.0,0.0,0.314286
44788,10.0,0.0,0.500000
312226,5.0,2.0,0.650000
...,...,...,...
30494,13.5,7.0,0.409836
263208,8.0,2.0,0.272727
283052,15.5,1.0,0.202532
176945,15.5,1.0,0.226667


In [61]:
def fuzzy_features(row):
    q1=row["question1"]
    q2=row["question2"]
    
    fuzzy_features=[0.0]*4
    
    fuzzy_features[0]=fuzz.QRatio(q1,q2)
    
    fuzzy_features[1]=fuzz.partial_ratio(q1,q2)
    
    fuzzy_features[2]=fuzz.token_set_ratio(q1,q2)
    
    fuzzy_features[3]=fuzz.token_sort_ratio(q1,q2)
    
    return fuzzy_features

In [62]:
fuzzy_features=df.apply(fuzzy_features,axis=1)

In [63]:
fuzzy_df=pd.DataFrame(fuzzy_features.to_list(),columns=["Qratio","Partial Ratio","Token Set Ratio","Token Sort Ratio"],index=df.index)

In [64]:
corpus=list(df["question1"])+list(df["question2"])
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(max_features=3000)
temp_df=pd.DataFrame(cv.fit_transform(corpus).toarray())
temp_df_1=temp_df[0:30000]
temp_df_1.index=new_df.index
temp_df_2=temp_df[30000:60000]
temp_df_2.index=new_df.index
bow_df=pd.concat([temp_df_1,temp_df_2],axis=1,ignore_index=True)

In [65]:
basic_features=df.drop(columns=["id","qid1","qid2","question1","question2","is_duplicate"])

In [66]:
final_df=pd.concat([bow_df,basic_features,new_df,length_features_df,fuzzy_df],axis=1)

In [67]:
final_df

,0,1,2,3,4,5,6,7,8,9,...,cmt_max,last_word_equal,first_word_equal,mean_length,abs_length,longest_substr_ratio,Qratio,Partial Ratio,Token Set Ratio,Token Sort Ratio
235234,0,0,0,0,0,0,0,0,0,0,...,0.315789,0.0,0.0,17.5,3.0,0.160494,32,41,52,45
322679,0,0,0,0,0,0,0,0,0,0,...,0.333333,0.0,0.0,6.5,5.0,0.400000,44,71,83,44
355018,0,0,0,0,0,0,0,0,0,0,...,0.624999,1.0,1.0,8.0,0.0,0.314286,81,79,84,81
44788,0,0,0,0,0,0,0,0,0,0,...,0.699999,0.0,0.0,10.0,0.0,0.500000,55,55,88,69
312226,0,0,0,0,0,0,0,0,0,0,...,0.499999,1.0,1.0,5.0,2.0,0.650000,72,74,91,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30494,0,0,0,0,0,0,0,0,0,0,...,0.352941,0.0,1.0,13.5,7.0,0.409836,61,77,70,58
263208,0,0,0,0,0,0,0,0,0,0,...,0.444444,1.0,1.0,8.0,2.0,0.272727,64,63,77,56
283052,0,0,0,0,0,0,0,0,0,0,...,0.500000,0.0,1.0,15.5,1.0,0.202532,59,67,77,62
176945,0,0,0,0,0,0,0,0,0,0,...,0.375000,0.0,1.0,15.5,1.0,0.226667,41,51,61,54


In [72]:
df

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q2_words,q1_words,word_common,total_words,word_share
235234,235234,345908,345909,i read that divorce rat in iran be increase do...,be there any significant differences in the di...,0,80,117,19,16,6,31,0.19
322679,322679,266746,448533,how do i focus,what should i do to focus on my education,0,14,41,9,4,3,13,0.23
355018,355018,41705,205197,why do rbi issue new 2k rupees note,why be the rbi issue 2k rupee note,1,35,34,8,8,5,16,0.31
44788,44788,80345,80346,how can the density of a sphere formula be cal...,what be the formula for the density of a sphere,0,52,47,10,10,7,19,0.37
312226,312226,73626,294927,why do star twinkle,why we can feel star twinkle,1,19,28,6,4,3,10,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30494,30494,56310,56311,be there any truth to the rothschild mh370 con...,be there any truth to these so call conspiracy...,0,60,94,17,10,6,26,0.23
263208,263208,12591,28646,what be the sole purpose of life,what do you feel be your purpose in life,1,32,40,9,7,4,16,0.25
283052,283052,86534,403086,what have be the single most important life de...,what be the biggest most important decision yo...,1,78,94,16,15,8,29,0.28
176945,176945,272126,272127,will china replace the unite state as the worl...,will the unite state of america surrive the lo...,0,74,88,16,15,6,27,0.22


In [68]:
final_df.columns= final_df.columns.astype(str)

In [69]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(final_df,df["is_duplicate"],test_size=0.05)

In [70]:
x_test


,0,1,2,3,4,5,6,7,8,9,...,cmt_max,last_word_equal,first_word_equal,mean_length,abs_length,longest_substr_ratio,Qratio,Partial Ratio,Token Set Ratio,Token Sort Ratio
52359,0,0,0,0,0,0,0,0,0,0,...,0.874999,0.0,1.0,7.5,1.0,0.962963,87,100,100,87
79644,0,0,0,0,0,0,0,0,0,0,...,0.111111,1.0,0.0,7.0,4.0,0.318182,48,62,44,51
162759,0,0,0,0,0,0,0,0,0,0,...,0.699999,1.0,1.0,9.0,2.0,0.404762,75,73,91,69
214731,0,0,0,0,0,0,0,0,0,0,...,0.400000,1.0,1.0,14.0,2.0,0.138889,57,54,59,51
290665,0,0,0,0,0,0,0,0,0,0,...,0.375000,0.0,1.0,11.5,9.0,0.486486,60,94,96,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397813,0,0,0,0,0,0,0,0,0,0,...,0.599999,0.0,0.0,5.0,0.0,0.666667,64,69,79,64
9533,0,0,0,0,0,0,0,0,0,0,...,0.444444,1.0,0.0,7.0,4.0,0.791667,63,83,88,63
107494,0,0,0,0,0,0,0,0,0,0,...,0.440000,0.0,1.0,24.5,1.0,0.126126,53,53,68,65
225719,0,0,0,0,0,0,0,0,0,0,...,0.812499,1.0,1.0,16.0,0.0,0.919192,94,93,97,93


In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf=RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred=rf.predict(x_test)

acs=accuracy_score(y_test,y_pred)
acs

0.796

In [73]:
def test_common_words(q1,q2):
    w1=set(map(lambda word: word.lower().strip(),q1.split()))
    w2=set(map(lambda word: word.lower().strip(),q2.split()))
    
    return len(w1&w2)
def test_total_words(q1,q2):
    w1=set(map(lambda word: word.lower().strip(),q1.split()))
    w2=set(map(lambda word: word.lower().strip(),q2.split()))
    
    return len(w1) +len(w2)

def test_fetch_token_features(q1,q2):
    
    
    STOPWORDS=stopwords.words("English")
    safe_div=0.00001
    
    #tokenizing both q1 and q2
    q1_tokens=q1.split()
    q2_tokens=q2.split()
    
    token_features=[0.0]*8
    
    if len(q1_tokens)==0 or len(q2_tokens)==0:
        return token_features
    
    #storing stop words of both q1 and q2
    q1_stopwords=set([word for word in q1_tokens if word in STOPWORDS])
    q2_stopwords=set([word for word in q2_tokens if word in STOPWORDS])
    
    #storing non-stopwords of both q1 and q2
    q1_words=set([word for word in q1_tokens if word not in STOPWORDS])
    q2_words=set([word for word in q2_tokens if word not in STOPWORDS])
    
    #common stop words
    common_stopwords=len(q1_stopwords&q2_stopwords)
    
    #common tokens
    common_tokens=len(set(q1_tokens)&set(q2_tokens))
    
    #common non stop words
    common_words=len(q1_words&q2_words)
    token_features[0]=common_words/(min(len(q1_words),len(q2_words))+safe_div)
    token_features[1]=common_words/(max(len(q1_words),len(q2_words))+safe_div)
    token_features[2]=common_stopwords/(min(len(q1_stopwords),len(q2_stopwords))+safe_div)
    token_features[3]=common_stopwords/(max(len(q1_stopwords),len(q2_stopwords))+safe_div)
    token_features[4]=common_tokens/(min(len(q1_tokens),len(q2_tokens))+safe_div)
    token_features[5]=common_tokens/(max(len(q1_tokens),len(q2_tokens))+safe_div)
    
    #checking if the last and first word are the same
    token_features[6]=int(q1_tokens[-1]==q2_tokens[-1])
    token_features[7]=int(q1_tokens[0]==q2_tokens[0])
    
    return token_features
def test_length_based_features(q1,q2):
    
    length_features=[0.0]*3
    
    q1_tokens=q1.split()
    q2_tokens=q2.split()
    
    if len(q1_tokens)==0 or len(q2_tokens)==0:
        return length_features
    length_features[0]=(len(q1_tokens)+len(q2_tokens))/2
    length_features[1]=abs(len(q1_tokens)-len(q2_tokens))
    
    substring=list(distance.lcsubstrings(q1,q2))
    if len(substring)==0:
        length_features[2]==0
    else:
        length_features[2]=len(substring[0])/(min(len(q1),len(q2))+1)
    
    return length_features
def test_fuzzy_features(q1,q2):
    
    fuzzy_features=[0.0]*4
    
    fuzzy_features[0]=fuzz.QRatio(q1,q2)
    
    fuzzy_features[1]=fuzz.partial_ratio(q1,q2)
    
    fuzzy_features[2]=fuzz.token_set_ratio(q1,q2)
    
    fuzzy_features[3]=fuzz.token_sort_ratio(q1,q2)
    
    return fuzzy_features
    

In [79]:
def query_point(q1,q2):
    query_input=[]
    
    q1=preprocessing(q1)
    q2=preprocessing(q2)
    
    q1bow=cv.transform([q1]).toarray()
    q2bow=cv.transform([q2]).toarray()
    
    query_input.append(len(q1))
    query_input.append(len(q2))
    query_input.append(len(q1.split()))
    query_input.append(len(q2.split()))
    query_input.append(test_common_words(q1,q2))
    query_input.append(test_total_words(q1,q2))
    query_input.append(test_common_words(q1,q2)/test_total_words(q1,q2))
    
    query_input.extend(test_fetch_token_features(q1,q2))
    query_input.extend(test_length_based_features(q1,q2))
    query_input.extend(test_fuzzy_features(q1,q2))
    
    
    
    return np.hstack((q1bow,q2bow,np.array(query_input).reshape(1,22)))
      
    
    
    
    
    
    
    
    

In [77]:
query_point("where is the capital of india?","what is the current capital of india?")

array([[29., 36.,  6., ...,  0.,  0.,  0.]])

In [80]:
rf.predict(query_point("where is the capital of india?","what is the current capital of india?"))

C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([1], dtype=int64)

In [33]:
import pickle

pickle.dump(rf,open("model.pkl","wb"))
pickle.dump(cv,open("cv.pkl","wb"))

In [42]:
import sklearn
sklearn.__version__

'1.2.1'